In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
import numpy.random as npr
import time
sys.path.append('../../src')
sys.path.append('../../interp')
import utils, lcp, train
from scipy.stats import ttest_ind


# sklearn models
from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import auc
from all_scores import get_scores

cred = (234/255, 51/255, 86/255)
cblue = (57/255, 138/255, 242/255)
out_dir = '../../results/interp_sim'
os.makedirs(out_dir, exist_ok=True)

Using TensorFlow backend.


# generate data

In [13]:
def gen_data(n=10, d=100, norm_beta=1, beta=None, var_eps=0.1, 
             s=None, seed=1, shift_type='None', shift_val=0.1):
    '''Generate data
    n : int
        number of samples
    d : int
        dimension
    norm_beta: float
        norm of beta
    var_eps: float
        variance of epsilon
        snr = norm_beta^2 / var_eps
    '''
    npr.seed(seed=seed)
    
    # x
    x = npr.randn(n, d)
    if 'shift' in shift_type:
        x += shift_val
    elif 'scale' in shift_type:
        S2 = np.cumsum(np.ones(d))
        S2 /= np.sum(S2)
        S2 = np.diag(np.sqrt(S2 * d))
        x = x @ S2
    elif 'spike' in shift_type:
        v = np.ones(d)
        v /= npl.norm(v)
        S = np.eye(d) + (np.sqrt(shift_val) - 1) * np.outer(v, v)
        x = x @ S
    elif shift_type == 'lap':
        x = npr.laplace(size=(n, d))
    
    # beta
    if s == None:
        s = d
    if beta is None:
        beta = np.zeros(d)
        beta[:s] = npr.randn(s)
        beta[:s] /= npl.norm(beta[:s])
        if norm_beta == 'd':
            norm_beta = d
        beta[:s] *= norm_beta
    
    var_mult = 0 if var_eps == 0 else np.sqrt(var_eps)
    eps = var_mult * npr.randn(n)
    y = x @ beta + eps

    return x, y, beta

In [14]:
n = 100
d = 10

beta = np.zeros(d)
beta[0] = 1
beta[1] = 2
# beta[important_vars] = 1
X, y, _ = gen_data(n=n, d=d, beta=beta, var_eps=0.5, 
             s=None, seed=1, shift_type='None', shift_val=0.1)
X_train, X_test, y_train, y_test = train_test_split(X, y) # defaults to 0.75: 0.25 split

In [15]:
# plt.plot(X[:, 0], y, 'o') 
# plt.show()

# fit a model to it

In [16]:
m = train.regress(X, y, model_type='linear')

In [17]:
scores = get_scores(m, X_train, X_test[0])

/accounts/projects/vision/.local/lib/python3.7/site-packages/shap/explainers/kernel.py:545: UserWarning: l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
  "l1_reg=\"auto\" is deprecated and in the next version (v0.29) the behavior will change from a " \


In [18]:
pd.DataFrame(scores).style.background_gradient(cmap='viridis')

,shap,lime,ice-contrib,ice-sensitivity
0,1.69235,3.49755,1.72624,1.0513
1,1.767,1.9359,2.09622,2.12779
2,0.0364153,0.140931,0.0242961,-0.0393479
3,0.019868,-0.116486,0.0412768,-0.0773394
4,-0.000135681,0.108711,-0.000284177,0.000530738
5,-0.0841483,0.0862501,-0.0919703,0.0561334
6,0.0689147,0.0539938,0.0699242,0.0748994
7,0.0415624,-0.0465102,0.0609753,0.0430812
8,0.0197551,0.0388597,0.0234699,0.0216637
9,0.0319245,-0.0294201,0.0166922,-0.0656815
